In [31]:
import pandas as pd
import glob
import re
import pickle

In [2]:
files = glob.glob('..\\..\\Normalizado\\Inferencia\\*.csv')
files

['..\\..\\Normalizado\\Inferencia\\Fortaleza.csv',
 '..\\..\\Normalizado\\Inferencia\\Joao_Pessoa.csv',
 '..\\..\\Normalizado\\Inferencia\\Maceio.csv',
 '..\\..\\Normalizado\\Inferencia\\Recife.csv',
 '..\\..\\Normalizado\\Inferencia\\Salvador.csv']

In [6]:
control = {}

for f in files:
    
    print(f)
    
    city = re.search(r'(\w+)\.csv$', f).group(1)
        
    df = pd.read_csv(f, sep=';', error_bad_lines=False)
    df['Cidade'] = city
    
    if city in control:
        
        control[city] = pd.concat([control[city], df], ignore_index = True)
        
    else:
    
        control[city] = df

..\..\Normalizado\Inferencia\Fortaleza.csv
..\..\Normalizado\Inferencia\Joao_Pessoa.csv
..\..\Normalizado\Inferencia\Maceio.csv
..\..\Normalizado\Inferencia\Recife.csv
..\..\Normalizado\Inferencia\Salvador.csv


In [7]:
control.keys()

dict_keys(['Salvador', 'Fortaleza', 'Recife', 'Joao_Pessoa', 'Maceio'])

In [8]:
for key in control:
    
    control[key] = control[key].tail(7)

In [10]:
control[list(control.keys())[0]]

,regiao,estado,municipio,coduf,codmun,codRegiaoSaude,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos,Recuperadosnovos,emAcompanhamentoNovos,interior/metropolitana,Cidade
120,Nordeste,BA,Salvador,29,292740,29020,SALVADOR,25/07/2020,30,2872347,52020,843,1636,6,NaN,NaN,1,Salvador
121,Nordeste,BA,Salvador,29,292740,29020,SALVADOR,26/07/2020,31,2872347,52263,243,1641,5,NaN,NaN,1,Salvador
122,Nordeste,BA,Salvador,29,292740,29020,SALVADOR,27/07/2020,31,2872347,52417,154,1649,8,NaN,NaN,1,Salvador
123,Nordeste,BA,Salvador,29,292740,29020,SALVADOR,28/07/2020,31,2872347,53520,1103,1655,6,NaN,NaN,1,Salvador
124,Nordeste,BA,Salvador,29,292740,29020,SALVADOR,29/07/2020,31,2872347,54488,968,1669,14,NaN,NaN,1,Salvador
125,Nordeste,BA,Salvador,29,292740,29020,SALVADOR,30/07/2020,31,2872347,55502,1014,1714,45,NaN,NaN,1,Salvador
126,Nordeste,BA,Salvador,29,292740,29020,SALVADOR,31/07/2020,31,2872347,56740,1238,1755,41,NaN,NaN,1,Salvador


In [27]:
data = {}

for key in control:

    sums = control[key].sum(axis = 0, skipna = True).to_dict()
    
    data[key] = {
        'deaths7perc': (sums["obitosNovos"] / control[key].iloc[0]["populacaoTCU2019"]) * 100,
        'cases7perc': (sums["casosNovos"] / control[key].iloc[0]["populacaoTCU2019"]) * 100,
        'deaths7Trend': (1, 0) [(control[key].iloc[-2]["obitosNovos"] - control[key].iloc[-1]["obitosNovos"]) > 0],
        'cases7Trend': (1, 0) [(control[key].iloc[-2]["casosNovos"] - control[key].iloc[-1]["casosNovos"]) > 0]
    }

C:\Users\DELL\.conda\envs\hackatonV2\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  # Remove the CWD from sys.path while we load stuff.
C:\Users\DELL\.conda\envs\hackatonV2\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  # This is added back by InteractiveShellApp.init_path()


In [28]:
data

{'Fortaleza': {'cases7Trend': 0,
  'cases7perc': 0.04753980568994157,
  'deaths7Trend': 1,
  'deaths7perc': 0.0014235718015900547},
 'Joao_Pessoa': {'cases7Trend': 1,
  'cases7perc': 0.20654746821752376,
  'deaths7Trend': 0,
  'deaths7perc': 0.007169211942918239},
 'Maceio': {'cases7Trend': 0,
  'cases7perc': 0.14878089951597137,
  'deaths7Trend': 1,
  'deaths7perc': 0.002747932181033772},
 'Recife': {'cases7Trend': 1,
  'cases7perc': 0.09503398801866896,
  'deaths7Trend': 0,
  'deaths7perc': 0.003524278328058056},
 'Salvador': {'cases7Trend': 1,
  'cases7perc': 0.19367437151569777,
  'deaths7Trend': 0,
  'deaths7perc': 0.004351841890969301}}

In [32]:
with open("..\\..\\Modelos\\regressaoV2.pkl","rb") as f:
    model = pickle.load(f)

In [33]:
model

AdaBoostRegressor(base_estimator=None, learning_rate=0.1, loss='linear',
         n_estimators=500, random_state=42)

In [41]:
predictions = {}

for city in data:
    
    print(city)
    
    datum = [data[city]["deaths7perc"], data[city]["cases7perc"], data[city]["deaths7Trend"], data[city]["cases7Trend"]]
    
    predictions[city] = model.predict([datum])
    
    print(datum)
    print(predictions[city])

Joao_Pessoa
[0.007169211942918239, 0.20654746821752376, 0, 1]
[12.0625]
Fortaleza
[0.0014235718015900547, 0.04753980568994157, 1, 0]
[23.]
Recife
[0.003524278328058056, 0.09503398801866896, 0, 1]
[12.0625]
Salvador
[0.004351841890969301, 0.19367437151569777, 0, 1]
[12.0625]
Maceio
[0.002747932181033772, 0.14878089951597137, 1, 0]
[23.]


In [45]:
from math import ceil

modelRMSE = 4.858189601618989

for city in predictions:
    
    print(city,":",ceil(predictions[city]),ceil(predictions[city] + 2*modelRMSE), ceil(predictions[city] + 3*modelRMSE))

Salvador : 13 22 27
Fortaleza : 23 33 38
Recife : 13 22 27
Joao_Pessoa : 13 22 27
Maceio : 23 33 38
